In [256]:
# Imports
import soundfile as sf
import numpy as np
import pyloudnorm as pyln

In [257]:
# get files in folder
import os
import glob

# SampleRate      
Fs = 48000

# Desired Wavetable size
numberOfFrames = 2
tableSize = 2048 * numberOfFrames

# Folder path to soundfiles
baseFolder = os.getcwd() + "/soundfiles"

# Output folder
outFolder = os.getcwd() + "/ouput"

# get all files in folder
files = glob.glob(baseFolder + "/*.wav")
numFiles = len(files)


In [258]:
# measure the loudness first 
meter = pyln.Meter(Fs) # create BS.1770 meter  

# clear output directory
def clearOutputFolder():
    for file in glob.glob(outFolder + "/*.wav"):
        os.remove(file)

# Pick random file from the file array
def pickRandomFile(fileArray):
    # Check if array is empty
    if len(fileArray) == 0:
        return "No files in folder"
    
    randomFile = np.random.randint(0, len(fileArray))
    return fileArray[randomFile]

# Read file and return data
def getFileData(Fs, file):
    # Check if file exists
    if not os.path.isfile(file):
        return "File does not exist"
    
    data, Fs = sf.read(file)
    return data

def fade(data, fadeLength, Fs):
    # check if data is empty
    if len(data) == 0:
        return "No audio data to fade"
    
    fade = np.linspace(1, 0, fadeLength)
    for i in range(data.shape[1]):  # apply fade to each channel
        data[:fadeLength, i] = data[:fadeLength, i] * fade
        data[-fadeLength:, i] = data[-fadeLength:, i] * fade[::-1]
    return data


In [259]:
# Write table to file, normalize to -12dB
def write_table(samplerate, audioData, audioLength, tableLength, tableName):
    # Check if audio data is empty
    if len(audioData) == 0:
        return "No audio data to write"
    
    if audioLength <= tableLength:
        raise ValueError("Audio Data length must be greater than the table data length. " 
                         "Possibly emptx audio data! "
                         "Do you have any audio files in your soundfiles folder? ")
    
    randomStart = np.random.randint(0, audioLength - tableLength)
    table = np.zeros(tableLength)
    
    for n in range(tableLength):
        table[n], buffer = audioData[randomStart + n]
    
    loudness = meter.integrated_loudness(audioData)
    pyln.normalize.loudness(table, loudness, -12.0)

    fade(audioData, 10, Fs)
    sf.write(tableName, table, samplerate)

# Finally we can generate the wavetables
def generate_experimental_wavetables(samplerate, numOutputFiles):
    # Check if folder exists
    if not os.path.isdir(outFolder):
        os.mkdir(outFolder)

    clearOutputFolder()
        
    file = pickRandomFile(files)
    audioData = getFileData(samplerate, file)
    numSamples = len(audioData)

    for i in range(numOutputFiles):
        write_table(Fs, audioData, numSamples, tableSize, 'output/table_'+str(i)+'.wav')

    


In [260]:
## Run the script
generate_experimental_wavetables(Fs, 10)